In [7]:
print(__doc__)

import numpy as np
import re
import json
import nltk
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')
    nltk.download("words")
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

from nltk.stem import SnowballStemmer
from nltk.tokenize import RegexpTokenizer

import csv

with open('emoji/emoji_final_join.json',encoding='utf-8') as f: 
    emoji_list = json.load(f)


stop_word = [line.rstrip('\r\n') for line in open("stop_words.txt")]

    

def split(word): 
    return list(word) 

def clean(text,emoji):
    am = text
    text = text.lower()
    
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split()) 
    text = text.split()

    newtext = [x for x in text if not x.startswith('@') and not x.startswith('https')]
    newtext = [x for x in text if x not in stop_word]
    newtext = [WordNetLemmatizer().lemmatize(x,'v') for x in newtext]
    newtext = ' '.join(newtext)
    
    tokenizer = RegexpTokenizer(r'\w+')
    newtext = tokenizer.tokenize(newtext)
    #stop_words = stopwords.words('english')
    #newtext =  [w for w in newtext if not w in stop_words]
    #stopwords.words will delete word 'not' which is important for sentiment analysis, so we don't remove stopwords.
    stemmer = SnowballStemmer('english')
    newtext = list(map(stemmer.stem, newtext))
    emojis = split(emoji)
    newtext = newtext + emojis
    label = 0
    for ele in newtext:
        if ele =='amp':
            label = 1

    newtext = ' '.join(newtext)
    if label == 1:
        newtext = newtext.replace('amp','and')
   
    #print(newtext)
    return newtext
    



with open('smalldata.csv',encoding="utf8") as words_file:
    csv_reader = csv.DictReader(words_file, delimiter = ',')
    data = []
    for row in csv_reader:
        cleaned_row = []
        clean_text = clean(row['Text'],row['Emoji'])
        
        if len(clean_text) == 0:
            continue
            
        if float(row['Sentiment']) > 0 and int(row['Sentiment140']) == 4:
            sentiment = 'positive'
        elif float(row['Sentiment']) < 0 and int(row['Sentiment140']) == 0:
            sentiment = 'negative'
        elif float(row['Sentiment']) == 0 and int(row['Sentiment140']) == 2:
            if float(row['Emoji_sentiment']) > 0:
                sentiment = 'positive'
            elif float(row['Emoji_sentiment']) < 0:
                sentiment = 'negative'
            else:
                sentiment = 'neutral'
        elif float(row['Sentiment']) == 0:
            if int(row['Sentiment140']) == 0:
                sentiment = 'negative'
            if int(row['Sentiment140']) == 4:
                sentiment = 'positive'
        elif int(row['Sentiment140']) == 2:
            if float(row['Sentiment']) < 0:
                sentiment = 'negative'
            if float(row['Sentiment']) > 0:
                sentiment = 'positive'
        else:
            continue
            
        
        cleaned_row.append(clean_text)
        cleaned_row.append(sentiment)
        #cleaned_row.append(row['Emoji'])
        data.append(np.array(cleaned_row))
    data = np.array(data)
    
print(data[:20])
print(len(data))

Automatically created module for IPython interactive environment


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/iris0301/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


[['realiz tonight titti man 😂 😎' 'positive']
 ['capper' 'neutral']
 ['actin like realli want gon get fuck good get feel 😋' 'positive']
 ['rememb first hear song alway favorit' 'positive']
 ['don t need cupcak ken cmon' 'neutral']
 ['ikyfl 😭' 'negative']
 ['cant go back sleep' 'negative']
 ['haha true 😂' 'positive']
 ['bounc shit forev knee' 'negative']
 ['btw devour blog day love back' 'positive']
 ['get forev onli god know much love mrsgarza' 'positive']
 ['alway felt should lion senat presid glad neither' 'positive']
 ['daniel tell drink more wine while alreadi kinda buzz' 'positive']
 ['don t do drug kid 🤧' 'negative']
 ['twitter get troubl parent idc sad' 'negative']
 ['don t even get fade anymor' 'negative']
 ['win t help california forest fire' 'neutral']
 ['probabl would ve die go astro fest' 'negative']
 ['can disappear no one would notic' 'neutral']
 ['funclejustin' 'neutral']]
1804


In [8]:
Y = data[:,1]
X = data[:,0]
print(Y)
print(X)

['positive' 'neutral' 'positive' ... 'neutral' 'neutral' 'positive']
['realiz tonight titti man 😂 😎' 'capper'
 'actin like realli want gon get fuck good get feel 😋' ...
 'quiero cerrar todo por un tiempo'
 'make comeback don t call comeback though' 'right lol']


In [22]:
word_dict = dict()
word_id = 0
for ele in X:
    ele = ele.split()
    for word in ele:
        if word not in word_dict:
            word_dict[word] = word_id 
            word_id += 1       
    
X_id = []
    
print(word_dict)

{'realiz': 0, 'tonight': 1, 'titti': 2, 'man': 3, '😂': 4, '😎': 5, 'capper': 6, 'actin': 7, 'like': 8, 'realli': 9, 'want': 10, 'gon': 11, 'get': 12, 'fuck': 13, 'good': 14, 'feel': 15, '😋': 16, 'rememb': 17, 'first': 18, 'hear': 19, 'song': 20, 'alway': 21, 'favorit': 22, 'don': 23, 't': 24, 'need': 25, 'cupcak': 26, 'ken': 27, 'cmon': 28, 'ikyfl': 29, '😭': 30, 'cant': 31, 'go': 32, 'back': 33, 'sleep': 34, 'haha': 35, 'true': 36, 'bounc': 37, 'shit': 38, 'forev': 39, 'knee': 40, 'btw': 41, 'devour': 42, 'blog': 43, 'day': 44, 'love': 45, 'onli': 46, 'god': 47, 'know': 48, 'much': 49, 'mrsgarza': 50, 'felt': 51, 'should': 52, 'lion': 53, 'senat': 54, 'presid': 55, 'glad': 56, 'neither': 57, 'daniel': 58, 'tell': 59, 'drink': 60, 'more': 61, 'wine': 62, 'while': 63, 'alreadi': 64, 'kinda': 65, 'buzz': 66, 'do': 67, 'drug': 68, 'kid': 69, '🤧': 70, 'twitter': 71, 'troubl': 72, 'parent': 73, 'idc': 74, 'sad': 75, 'even': 76, 'fade': 77, 'anymor': 78, 'win': 79, 'help': 80, 'california': 81

In [26]:

all_dict = []

for i in range(len(X)):
    wordDict = dict.fromkeys(word_dict, 0)
    ele = X[i].split()
    for word in ele:
        wordDict[word] += 1
    all_dict.append(wordDict)

[]


In [28]:
Xnew = []
for i in range(len(X)):
    ele = X[i].split()
    Xnew.append(ele)
print(Xnew[:10])

[['realiz', 'tonight', 'titti', 'man', '😂', '😎'], ['capper'], ['actin', 'like', 'realli', 'want', 'gon', 'get', 'fuck', 'good', 'get', 'feel', '😋'], ['rememb', 'first', 'hear', 'song', 'alway', 'favorit'], ['don', 't', 'need', 'cupcak', 'ken', 'cmon'], ['ikyfl', '😭'], ['cant', 'go', 'back', 'sleep'], ['haha', 'true', '😂'], ['bounc', 'shit', 'forev', 'knee'], ['btw', 'devour', 'blog', 'day', 'love', 'back']]


In [60]:
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf



idfs = computeIDF(all_dict)
result = []
for i in range(len(all_dict)):
    tf = computeTF(all_dict[i], Xnew[i])
    tfidf = computeTFIDF(tf, idfs)
    output = list(tfidf.values())
    result.append(output)


document_topic_matrix = TruncatedSVD(n_components=100, random_state=0).fit_transform(result)

id_topic={}
a = np.argsort(document_topic_matrix, axis=1)
for i in range(len(document_topic_matrix)):
    top5 = np.argsort(document_topic_matrix[i])[-5:][::-1]
    id_topic[i] = list(top5)


X_new = list(id_topic.values())

In [38]:
print(X_new[:10])

[[0, 9, 97, 80, 96], [43, 25, 60, 42, 23], [80, 97, 9, 89, 13], [13, 9, 4, 92, 0], [80, 9, 69, 89, 93], [3, 0, 78, 76, 73], [80, 9, 87, 5, 95], [0, 86, 88, 9, 5], [13, 80, 94, 95, 90], [9, 65, 56, 92, 66]]


In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, Y, test_size=0.2, shuffle = True)


In [85]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
decision_tree = DecisionTreeClassifier(random_state=0)
decision_tree.fit(X_train, y_train)
decision_tree_score = decision_tree.score(X_test, y_test)
print('Decision Tree:', decision_tree_score)

Decision Tree: 0.5429362880886427


In [86]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_score = knn.score(X_test, y_test)
print('KNN:',knn_score)

KNN: 0.481994459833795


In [87]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
svm = SVC(random_state=0)
svm.fit(X_train, y_train)
svm_score = svm.score(X_test, y_test)
print('SVM:',svm_score)

SVM: 0.5263157894736842


/Users/iris0301/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [144]:
from gensim.models.phrases import Phrases, Phraser

tokenized_train = [t.split() for t in X]
phrases = Phrases(tokenized_train)
bigram = Phraser(phrases)
tg_phrases = Phrases(bigram[tokenized_train])
trigram = Phraser(tg_phrases)

In [155]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle = True)

In [148]:
import pandas as pd
from gensim.models.doc2vec import LabeledSentence, Doc2Vec, TaggedDocument

def labelize_tweets_bg(tweets,label):
    result = []
    prefix = label
    for i, t in enumerate(tweets):
        result.append(LabeledSentence(trigram[bigram[t.split()]], [prefix + '_%s' % i]))
    return result
  
#all_x = pd.concat([X_train,X_test])
all_x_w2v_bg = labelize_tweets_bg(X, 'all')


/Users/iris0301/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning:

Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).



In [149]:
from sklearn import utils
from tqdm import tqdm

#cores = multiprocessing.cpu_count()
model_bg_dbow = Doc2Vec(dm=0, size=100, negative=5, min_count=2, alpha=0.065, min_alpha=0.065)
model_bg_dbow.build_vocab([x for x in tqdm(all_x_w2v_bg)])

for epoch in range(30):
    model_bg_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v_bg)]), total_examples=len(all_x_w2v_bg), epochs=1)
    model_bg_dbow.alpha -= 0.002
    model_bg_dbow.min_alpha = model_bg_dbow.alpha
    

/Users/iris0301/opt/anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning:

The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.

100%|██████████| 1804/1804 [00:00<00:00, 2458260.04it/s]


In [150]:
def get_vectors(model, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in range(len(corpus)):
        prefix = 'all_' + str(i)
        vecs[n] = model.docvecs[prefix]
        n += 1
    return vecs

train_vecs_dbow_bg = get_vectors(model_bg_dbow, X_train, 100)
validation_vecs_dbow_bg = get_vectors(model_bg_dbow, X_test, 100)


In [156]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(train_vecs_dbow_bg, y_train)
clf.score(validation_vecs_dbow_bg, y_test)

/Users/iris0301/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

/Users/iris0301/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



0.4903047091412742